CONEXÂO COM POSTGRESQL E REALIZAÇÃO DOS EXERCÍCIOS

In [ ]:
#Installing libraries for the connection 
# !pip install sqlalchemy
# !pip install psycopg2

In [4]:
#imports
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from pandasql import sqldf


In [3]:
#PROJETO VERSÃO SEM O POSTGRES DEVIDO A PROBLEMAS DE IMPORTAÇÃO DA BASE DE DADOS

In [150]:
#criando a base de dados a partir de dataframe do pandas lido de um .csv

br_dataset = pd.read_csv(r'C:\Users\eduar\Documents\ADA\sql\projeto\datasets\BR_youtube_trending_data.csv')


#1 - Qual a categoria de video mais assistida no Brasil?
query = '''
--SQL code here
SELECT
    *
FROM
    br_dataset
LIMIT 10

'''
#chamada da query sql
result = sqldf(query, locals())
result





#Lidando com o json de categories para criar uma tupla de id,categorias convertida em dataframe (a leitura direta do sjon não funciona como esperado)

categories_df = pd.read_json(r'C:\Users\eduar\Documents\ADA\sql\projeto\datasets\br_category_id.json')
category_list = []
for description in list(categories_df['items'].items()):
   
    category = description[1]['snippet']['title']
    category_list.append(category)

category_list

index = [i for i in range(len(category_list))]
index = tuple(zip(index,category_list))
categories_df = pd.DataFrame(index)
categories_df.rename(columns={0:'categoryId',1:'category'},inplace=True)
categories_df

,categoryId,category
0,0,Film & Animation
1,1,Autos & Vehicles
2,2,Music
3,3,Pets & Animals
4,4,Sports
5,5,Short Movies
6,6,Travel & Events
7,7,Gaming
8,8,Videoblogging
9,9,People & Blogs


In [28]:
#1 - Quais as categorias mais presentes nos top trendings do Brasil?
query = '''
--SQL code here
SELECT
    categoryid,
    COUNT(categoryid) AS numero_de_videos
FROM
    br_dataset
GROUP BY categoryid
ORDER BY COUNT(categoryid) DESC


'''
#chamada da query sql
result = sqldf(query, locals())
result

,categoryId,numero_de_videos
0,24,63442
1,10,57021
2,17,39530
3,20,39362
4,22,31894
5,23,10257
6,25,4420
7,28,3672
8,1,3207
9,27,3120


In [29]:
#2 - Quais os canais mais presentes nos top trendings do Brasil?
query = '''
--SQL code here
SELECT
    channelTitle,
    COUNT(channelTitle) AS videos_do_canal
FROM
    br_dataset
GROUP BY channelTitle
ORDER BY COUNT(channelTitle) DESC


'''
#chamada da query sql
result = sqldf(query, locals())
result

,channelTitle,videos_do_canal
0,ge,3049
1,Felipe Neto,2610
2,ESPN Brasil,2397
3,Jazzghost,2120
4,Enaldinho,1902
...,...,...
5708,Centro de Mídias SP,1
5709,CNN Brasil Economia,1
5710,CANAL DO BAROLO 2,1
5711,Brandon & Bruna,1


In [106]:
#3 -  Quais os canais mais hegemônicos dentro de suas categorias
#Dada uma categoria, qual o canal mais presente -  se der ver quantos por cento e filtrar por qual a maior porcentagem pra saber qual tem mais controle da sua categoria dentre todas as categorias



query = '''
--SQL code here
SELECT
    categoryid AS id_da_categoria,
    channelTitle AS canal_mais_visto,
    view_count AS numero_de_views
FROM
(SELECT 
    categoryid,
    channelTitle,
    (SUM(view_count)) AS view_count
FROM br_dataset
GROUP BY channelTitle
ORDER BY categoryid,SUM(view_count) DESC

) AS tabela

GROUP BY categoryid
ORDER BY view_count DESC




'''




#chamada da query sql
result = sqldf(query, locals())
result  

,id_da_categoria,canal_mais_visto,numero_de_views
0,24,MrBeast,21365154239
1,10,HYBE LABELS,15259144198
2,20,Brawl Stars,3614488781
3,17,ESPN Brasil,2991688085
4,23,Enaldinho,2576353487
5,22,Problems,1875468244
6,28,Apple,1762752932
7,1,Alan Becker,800014689
8,27,YouTube,675030926
9,25,Band Jornalismo,410557567


In [ ]:
pd.read_sql_query(query, con = conn)